# Chainer Notebook
* ChainerでMNISTサンプルの学習・推論を行う備忘録。
* 必要なライブラリを読み込む

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training , utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
from chainer import computational_graph
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from sklearn.datasets import fetch_mldata

* データセットをロードする

In [2]:
mnist = fetch_mldata('MNIST original')

# 70,000件の784次元ベクトルデータ
mnist.data = mnist.data.astype(np.float32)
mnist.data /= 255 # 0-1のデータに変換

# 正解データ(教師データ)
mnist.target = mnist.target.astype(np.int32)

* 学習用データと検証用データに分割する

In [3]:
N = 60000 # 学習データ数
x_train, x_test = np.split(mnist.data, [N])
y_train, y_test = np.split(mnist.target, [N])
N_test = y_test.size # 検証用データ数

### 多層パーセプトロン(MLP)による学習
* 各パラメータの設定

In [4]:
batch_size = 100
n_epoch = 20
n_units = 1024

* モデルを定義する

In [5]:
class MLP(chainer.Chain):
    
    def __init__(self, n_in, n_units, n_out):
        super(MLP, self).__init__(
            l1=L.Linear(n_in, n_units),    # 入力層
            l2=L.Linear(n_units, n_units), # 中間層
            l3=L.Linear(n_units, n_out),   # 出力層
        )
        
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

最終層の活性関数として, softmax関数を使用し、損失関数として、交差エントロピー関数で評価する。

In [6]:
model = L.Classifier(MLP(784, n_units, 10))

* 最適化手法の選択

In [7]:
optimizer = optimizers.Adam()
optimizer.setup(model)

* 学習

In [8]:
def plot():
    with open('graph.dot', 'w') as o:
        variable_style = {'shape': 'octagon', 'fillcolor': '#E0E0E0',
                                  'style': 'filled'}
        function_style = {'shape': 'record', 'fillcolor': '#6495ED',
                                  'style': 'filled'}
        g = computational_graph.build_computational_graph(
            (model.loss,),
            variable_style=variable_style,
            function_style=function_style)
        o.write(g.dump())
        
    print('graph generated')

In [9]:
import six
import time
for epoch in six.moves.range(1, n_epoch + 1):
    print('epoch', epoch)
    
    # training
    perm = np.random.permutation(N) # データをランダムに並び替え
    sum_accuracy = 0
    sum_loss = 0
    start = time.time()
    for i in six.moves.range(0, N, batch_size):
        x = chainer.Variable(np.asarray(x_train[perm[i:i + batch_size]]))
        t = chainer.Variable(np.asarray(y_train[perm[i:i + batch_size]]))
        
        optimizer.update(model, x, t)
        if epoch == 1 and i == 0:
            plot()
        
        # len(t.data)はbatch_sizeと置き換えても同じ
        sum_loss += float(model.loss.data) * len(t.data)
        sum_accuracy += float(model.accuracy.data) * len(t.data)

end = time.time()
elapsed_time = end - start
throughput = N / elapsed_time
print('train mean loss={}, accuracy={}, throughput={} images/sec'.format(
       sum_loss / N, sum_accuracy / N, throughput))

('epoch', 1)
graph generated
('epoch', 2)
('epoch', 3)
('epoch', 4)
('epoch', 5)
('epoch', 6)
('epoch', 7)
('epoch', 8)
('epoch', 9)
('epoch', 10)
('epoch', 11)
('epoch', 12)
('epoch', 13)
('epoch', 14)
('epoch', 15)
('epoch', 16)
('epoch', 17)
('epoch', 18)
('epoch', 19)
('epoch', 20)
train mean loss=0.00702655764213, accuracy=0.997983335157, throughput=752.208425374 images/sec


* 学習したモデルを評価する

In [10]:
sum_accuracy = 0
sum_loss = 0
for i in six.moves.range(0, N_test, batch_size):
    x = chainer.Variable(np.asarray(x_test[i:i + batch_size]), volatile='on')
    t = chainer.Variable(np.asarray(y_test[i:i + batch_size]), volatile='on')
    
    loss = model(x, t)
    sum_loss += float(loss.data) * len(t.data)
    sum_accuracy += float(model.accuracy.data) * len(t.data)
    
print('test mean loss={}, accuracy={}'.format(sum_loss / N_test, sum_accuracy / N_test))

test mean loss=0.0861607757364, accuracy=0.985400004983


* モデルをnpz形式で保存する

In [11]:
serializers.save_npz('mlp.model', model) # モデル
serializers.save_npz('mlp.state', optimizer) # 学習状況

### 多層パーセプトロン(MLP)による学習
* モデルは前と変わらず
* v1.11.0で実装されたTrainerを使用
* データの読み込みについてもスッキリ書き換えられている
    * 60,000は学習データに、10,000は評価データに分けられている

In [12]:
train, test = chainer.datasets.get_mnist()

train_iter = chainer.iterators.SerialIterator(train, batch_size) # エポック毎の学習データの並び替えもやってくれる
test_iter = chainer.iterators.SerialIterator(test, batch_size, repeat=False, shuffle=False) # repeat=Flaseがないと無限ループ

* Trainerの設定

In [13]:
updater = training.StandardUpdater(train_iter, optimizer)
trainer = training.Trainer(updater, (n_epoch, 'epoch'), out='result')

* 試験データを用いた評価を設定

In [14]:
trainer.extend(extensions.Evaluator(test_iter, model))

* 計算グラフのダンプを設定

In [15]:
trainer.extend(extensions.dump_graph('main/loss'))

* スナップショットをエポック毎に取得するよう設定

In [17]:
trainer.extend(extensions.snapshot())

* エポック毎に学習状況を標準出力するよう設定

In [19]:
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy']))

* 進捗をバー表示する

In [20]:
trainer.extend(extensions.ProgressBar())

* 学習の実行

In [21]:
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
     total [..................................................]  1.67%
this epoch [################..................................] 33.33%
       200 iter, 0 epoch / 20 epochs
    4.7132 iters/sec. Estimated time to finish: 0:41:43.594672.
     total [#.................................................]  2.50%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 20 epochs
    4.7261 iters/sec. Estimated time to finish: 0:41:15.618736.
     total [#.................................................]  3.33%
this epoch [#################################.................] 66.67%
       400 iter, 0 epoch / 20 epochs
    4.7107 iters/sec. Estimated time to finish: 0:41:02.453220.
     total [##................................................]  4.17%
this epoch [#########################################.........] 83.33%
       500 iter, 0 epoch / 20 epochs
    4.

## 畳込みニューラルネットワーク(CNN)によるMNISTの学習